# Big $n$ regression

Here is an experience to do linear regression on a dataset with more than 100 million observations. I learnt various methods for solving linear regression and realize that, with right choice of algorithm, it is a problem that can be handled by any moderate computer.

### Q1(1)

Download the flight data from <http://stat-computing.org/dataexpo/2009/the-data.html>. For this exercise, we only need data from years 2003-2008. If you are using Mac or Linux, you can run the following Bash script, which downloads and unzips files for all years.
```bash
# Download flight data by year
for i in {1987..2008}
  do
    echo "$(date) $i Download"
    fnam=$i.csv.bz2
    wget -O ./$fnam http://stat-computing.org/dataexpo/2009/$fnam
    echo "$(date) $i unzip"
    bzip2 -d ./$fnam
  done

# Download airline carrier data
wget -O ./airlines.csv http://www.transtats.bts.gov/Download_Lookup.asp?Lookup=L_UNIQUE_CARRIERS

# Download airports data
wget -O ./airports.csv https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
```
Find out how many data points in each year.

In [7]:
range=[2003,2004,2005,2006,2007,2008]

for i in range
    @show string(i,".csv")
    @show countlines(string(i,".csv"))
end

string(i, ".csv") = "2003.csv"
countlines(string(i, ".csv")) = 6488541
string(i, ".csv") = "2004.csv"
countlines(string(i, ".csv")) = 7129271
string(i, ".csv") = "2005.csv"
countlines(string(i, ".csv")) = 7140597
string(i, ".csv") = "2006.csv"
countlines(string(i, ".csv")) = 7141923
string(i, ".csv") = "2007.csv"
countlines(string(i, ".csv")) = 7453216
string(i, ".csv") = "2008.csv"
countlines(string(i, ".csv")) = 7009729


So from above we find the year 2003-2008 files have 6488541, 7129271, 7140597, 7141923, 7453216, 7009729 lines respectively.

### Q1(2) 

We are interested in how the time gain of a flight, defined as `DepDelay - ArrDelay`, depends on the distance traveled (`Distance`), departure delay (`DepDelay`), and carrier (`UniqueCarrier`). 

We want to fit a linear regression `Gain ~ 1 + Distance + DepDelay + UniqueCarrier` using data from 2003-2008. Note `UniqueCarrier` is a factor with 23 levels: "9E", "AA", "AQ", "AS", "B6", "CO", "DH", "DL", "EV", "F9", "FL", "HA", "HP", "MQ", "NW", "OH", "OO", "TZ", "UA", "US", "WN", "XE", and "YV". We use the dummy coding with "9E" as base level.

Will the design matrix (in double precision) fit into the memory of you computer?

In [1]:
# import data from csv
using JuliaDB

yrtable = loadtable(
    "2003.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])

Table with 6488540 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
-4        -1        "UA"           837
-1        -3        "UA"           837
29        23        "UA"           837
-2        -9        "UA"           1835
18        52        "UA"           1835
-4        6         "UA"           1835
-4        -8        "UA"           1835
0         2         "UA"           1835
-4        19        "UA"           1835
3         4         "UA"           413
-4        -23       "UA"           413
-3        -19       "UA"           413
⋮
#NA       #NA       "DL"           1891
29        62        "DL"           581
39        66        "DL"           1891
26        27        "DL"           1678
114       134       "DL"           946
44        53        "DL"           813
16        47        "DL"           432
50        54        "DL"           432
-3        -5        "DL"           453
3         3         "DL"           689
-1     

In [2]:
# drop rows with missing values
yrtable = dropna(yrtable)

Table with 6375689 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
-4        -1        "UA"           837
-1        -3        "UA"           837
29        23        "UA"           837
-2        -9        "UA"           1835
18        52        "UA"           1835
-4        6         "UA"           1835
-4        -8        "UA"           1835
0         2         "UA"           1835
-4        19        "UA"           1835
3         4         "UA"           413
-4        -23       "UA"           413
-3        -19       "UA"           413
⋮
70        66        "DL"           1891
29        62        "DL"           581
39        66        "DL"           1891
26        27        "DL"           1678
114       134       "DL"           946
44        53        "DL"           813
16        47        "DL"           432
50        54        "DL"           432
-3        -5        "DL"           453
3         3         "DL"           689
-1     

In [3]:
whos(r"yrtable")

                       yrtable 155656 KB     IndexedTables.NextTable{IndexedTab…


In [1]:
# import data from csv
using JuliaDB

# only need columns: DepDelay, ArrDelay, UniqueCarrier, Distance
yrtable1 = loadtable(
    "2003.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable1 = dropna(yrtable1)
    
yrtable2 = loadtable(
    "2004.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable2 = dropna(yrtable2)

yrtable3 = loadtable(
    "2005.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable3 = dropna(yrtable3)

yrtable4 = loadtable(
    "2006.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable4 = dropna(yrtable4)

yrtable5 = loadtable(
    "2007.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable5 = dropna(yrtable5)

yrtable6 = loadtable(
    "2008.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable6 = dropna(yrtable6)

y_big = [yrtable1;yrtable2;yrtable3;yrtable4;yrtable5;yrtable6]
save(y_big, "y_big.csv")

6-element Array{IndexedTables.NextTable{IndexedTables.Columns{NamedTuples._NT_DepDelay_ArrDelay_UniqueCarrier_Distance{Int64,Int64,String,Int64},NamedTuples._NT_DepDelay_ArrDelay_UniqueCarrier_Distance{Array{Int64,1},Array{Int64,1},PooledArrays.PooledArray{String,UInt8,1,Array{UInt8,1}},Array{Int64,1}}}},1}:
 Table with 6375689 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
-4        -1        "UA"           837
-1        -3        "UA"           837
29        23        "UA"           837
-2        -9        "UA"           1835
18        52        "UA"           1835
-4        6         "UA"           1835
-4        -8        "UA"           1835
0         2         "UA"           1835
-4        19        "UA"           1835
3         4         "UA"           413
-4        -23       "UA"           413
-3        -19       "UA"           413
⋮
70        66        "DL"           1891
29        62        "DL"           581
39        

In [2]:
whos(r"y_big")

                         y_big 1012951 KB     6-element Array{IndexedTables.Nex…


we can see above that just the table itself is this big (~990Mb), and the above table has only four columns. The design thable will have the other 23 extended columns to indicate the flight carrier. So probably will be around 990Mb*23. Therefore the RAM of my computer won't be enough. 

### Q1(3)

Review the [Summary of Linear Regression](http://hua-zhou.github.io/teaching/biostatm280-2018spring/slides/12-linreg/linreg.html) and devise a strategy to solve the linear regression.

Report the estimated regression coefficients $\widehat \beta$, estimated variance $\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - 1)$, and coefficient standard errors.

Hint: It took my laptop less than 3 minutes to import data and fit linear regression.

In [22]:
using JuliaDB
yrtable1 = loadtable(
    "2003.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable1 = dropna(yrtable1)
    
yrtable2 = loadtable(
    "2004.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable2 = dropna(yrtable2)

yrtable3 = loadtable(
    "2005.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable3 = dropna(yrtable3)

yrtable4 = loadtable(
    "2006.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable4 = dropna(yrtable4)

yrtable5 = loadtable(
    "2007.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable5 = dropna(yrtable5)

yrtable6 = loadtable(
    "2008.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
yrtable6 = dropna(yrtable6)

Table with 6855029 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
8         -14       "WN"           810
19        2         "WN"           810
8         14        "WN"           515
-4        -6        "WN"           515
34        34        "WN"           515
25        11        "WN"           688
67        57        "WN"           1591
-1        -18       "WN"           1591
2         2         "WN"           451
0         -16       "WN"           451
6         1         "WN"           828
94        80        "WN"           828
⋮
57        75        "DL"           481
80        99        "DL"           689
-2        15        "DL"           270
-4        6         "DL"           425
-3        16        "DL"           546
-1        2         "DL"           215
3         14        "DL"           533
-1        -2        "DL"           874
-5        0         "DL"           545
11        9         "DL"           533
7         -5  

In [53]:
xy1 = generate_xy(yrtable1)
xy2 = generate_xy(yrtable2)
xy3 = generate_xy(yrtable3)
xy4 = generate_xy(yrtable4)
xy5 = generate_xy(yrtable5)
xy6 = generate_xy(yrtable6)

G1 = xy1' * xy1
G2 = xy2' * xy2
G3 = xy3' * xy3
G4 = xy4' * xy4
G5 = xy5' * xy5
G6 = xy6' * xy6

G = G1+G2+G3+G4+G5+G6

26×26 Array{Float64,2}:
      4.14904e7   3.00028e10  …  822290.0              6.0236e7  
      3.00028e10  3.51353e13          3.27927e8        5.75882e10
      3.69821e8   2.76146e11          1.0596e7         9.8893e7  
      3.91626e6   4.19149e9           0.0              3.57932e6 
  88336.0         3.75197e7           0.0         111437.0       
 937547.0         8.30881e8   …       0.0              1.45831e6 
 799117.0         9.55599e8           0.0              1.2441e6  
      1.8094e6    2.03781e9           0.0         570038.0       
 669687.0         2.51484e8           0.0              1.88422e6 
      3.37715e6   2.9514e9            0.0              1.01046e6 
      1.64509e6   7.42218e8   …       0.0              4.54593e6 
 334842.0         2.97744e8           0.0         128138.0       
      1.24932e6   8.33171e8           0.0         957032.0       
 272859.0         1.61374e8           0.0          68090.0       
 574927.0         5.84803e8           0.0           

In [54]:
# Pkg.add("SweepOperator")
using SweepOperator
sweep_result = sweep!(G, 1:25)

26×26 Array{Float64,2}:
     -2.00254e-6   4.21792e-11  …   1.98347e-6    1.14033   
      3.00028e10  -9.38376e-14     -5.01596e-12   0.00164935
      3.69821e8    2.76146e11       1.16836e-10  -0.0118811 
      3.91626e6    4.19149e9       -1.97927e-6   -1.8723    
  88336.0          3.75197e7       -1.98134e-6   -0.5789    
 937547.0          8.30881e8    …  -1.98009e-6   -0.938452  
 799117.0          9.55599e8       -1.97878e-6   -1.42247   
      1.8094e6     2.03781e9       -1.97887e-6   -2.57627   
 669687.0          2.51484e8       -1.98271e-6    1.16808   
      3.37715e6    2.9514e9        -1.97993e-6   -2.19625   
      1.64509e6    7.42218e8    …  -1.98278e-6    1.03932   
 334842.0          2.97744e8       -1.97972e-6   -2.15207   
      1.24932e6    8.33171e8       -1.98132e-6   -1.35247   
 272859.0          1.61374e8       -1.98044e-6   -1.87248   
 574927.0          5.84803e8       -1.97917e-6   -0.350758  
      2.90986e6    1.13812e9    …  -1.98262e-6   -1.46395   


In [55]:
#coefficient
beta =  sweep_result[1:25,26]

25-element Array{Float64,1}:
  1.14033   
  0.00164935
 -0.0118811 
 -1.8723    
 -0.5789    
 -0.938452  
 -1.42247   
 -2.57627   
  1.16808   
 -2.19625   
  1.03932   
 -2.15207   
 -1.35247   
 -1.87248   
 -0.350758  
 -1.46395   
 -3.62506   
 -0.00722279
 -0.40365   
 -3.5774    
 -1.14816   
 -0.883753  
  2.74855   
 -2.56721   
 -0.202211  

In [65]:
#estimated variance
sigma_square =  sweep_result[26,26]/BigInt(6488541+7129271+7140597+7141923+7453216+7009729-1)


2.001913490791941147215896897940565314165032940323123263649392931745882919913941e+02

coefficient standard errors
$$var(\hat{\beta})=\hat{\sigma}^2(X'X)^{-1}$$

In [71]:
#coefficient standard errors
cse = sigma_square * (-sweep_result[1,1])

4.008904970143334892251028546440452915612160145754481487935512324851734615550728e-04

### Q1(4)

Go to your resume/cv and claim you have experience performing analytics on data with hundred millions of observations.

### Sample code

Following code explores the data in 2003 and generates the design matrix and responses for that year. Feel free to use the code in your solution.

In [1]:
;head 2003.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2003,1,29,3,1651,1655,1912,1913,UA,1017,N202UA,141,138,119,-1,-4,ORD,MSY,837,5,17,0,NA,0,NA,NA,NA,NA,NA
2003,1,30,4,1654,1655,1910,1913,UA,1017,N311UA,136,138,108,-3,-1,ORD,MSY,837,2,26,0,NA,0,NA,NA,NA,NA,NA
2003,1,31,5,1724,1655,1936,1913,UA,1017,N317UA,132,138,110,23,29,ORD,MSY,837,5,17,0,NA,0,NA,NA,NA,NA,NA
2003,1,1,3,1033,1035,1625,1634,UA,1018,N409UA,232,239,215,-9,-2,OAK,ORD,1835,6,11,0,NA,0,NA,NA,NA,NA,NA
2003,1,2,4,1053,1035,1726,1634,UA,1018,N496UA,273,239,214,52,18,OAK,ORD,1835,13,46,0,NA,0,NA,NA,NA,NA,NA
2003,1,3,5,1031,1035,1640,1634,UA,1018,N412UA,249,239,223,6,-4,OAK,ORD,1835,13,13,0,NA,0,NA,NA,NA,NA,NA
2003,1,4,6,1031,1035,1626,1634,UA,1018,N455UA,235,239,219,-8,-4,OAK,ORD,183

In [2]:
# how many data points
countlines("2003.csv")

6488541

In [32]:
# import data from csv
using JuliaDB
# only need columns: DepDelay, ArrDelay, UniqueCarrier, Distance
@time yrtable = loadtable(
    "2003.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])

 54.646481 seconds (144.01 M allocations: 7.497 GiB, 24.77% gc time)


Table with 6488540 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
-4        -1        "UA"           837
-1        -3        "UA"           837
29        23        "UA"           837
-2        -9        "UA"           1835
18        52        "UA"           1835
-4        6         "UA"           1835
-4        -8        "UA"           1835
0         2         "UA"           1835
-4        19        "UA"           1835
3         4         "UA"           413
-4        -23       "UA"           413
-3        -19       "UA"           413
⋮
#NA       #NA       "DL"           1891
29        62        "DL"           581
39        66        "DL"           1891
26        27        "DL"           1678
114       134       "DL"           946
44        53        "DL"           813
16        47        "DL"           432
50        54        "DL"           432
-3        -5        "DL"           453
3         3         "DL"           689
-1     

In [33]:
# drop rows with missing values
yrtable = dropna(yrtable)

Table with 6375689 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
-4        -1        "UA"           837
-1        -3        "UA"           837
29        23        "UA"           837
-2        -9        "UA"           1835
18        52        "UA"           1835
-4        6         "UA"           1835
-4        -8        "UA"           1835
0         2         "UA"           1835
-4        19        "UA"           1835
3         4         "UA"           413
-4        -23       "UA"           413
-3        -19       "UA"           413
⋮
70        66        "DL"           1891
29        62        "DL"           581
39        66        "DL"           1891
26        27        "DL"           1678
114       134       "DL"           946
44        53        "DL"           813
16        47        "DL"           432
50        54        "DL"           432
-3        -5        "DL"           453
3         3         "DL"           689
-1     

In [51]:
# mapping from variable names to X columns
# carrier "9E" is used as base level
const var2col = Dict(
        "Intercept" => 1,
        "Distance" => 2,
        "DepDelay" => 3,
        "AA" => 4,
        "AQ" => 5,
        "AS" => 6,
        "B6" => 7,
        "CO" => 8,
        "DH" => 9,
        "DL" => 10,
        "EV" => 11,
        "F9" => 12,
        "FL" => 13,
        "HA" => 14,
        "HP" => 15,
        "MQ" => 16,
        "NW" => 17,
        "OH" => 18,
        "OO" => 19,
        "TZ" => 20,
        "UA" => 21,
        "US" => 22,
        "WN" => 23,
        "XE" => 24,
        "YV" => 25,
        "Gain" => 26)
# mapping from column to variable names
const col2var = map(reverse, var2col)

# a custom function to generate [X y] from data table
function generate_xy(tbl::NextTable)
    # X matrix
    XY = zeros(length(tbl), 26)
    # intercept term
    @views fill!(XY[:, 1], 1)
    # Distance term
    @views copy!(XY[:, 2], columns(tbl, :Distance))
    # DepDelay term
    @views copy!(XY[:, 3], columns(tbl, :DepDelay))
    # Dummy coding for airline
    @inbounds for i in 1:length(tbl)
        tbl[i][:UniqueCarrier] == "9E" && continue # base level
        XY[i, var2col[tbl[i][:UniqueCarrier]]] = 1
    end
    # last column is response: gain = depdelay - arrdelay
    XY[:, 26] = select(tbl, 
        (:DepDelay, :ArrDelay) => p -> Float64(p.DepDelay - p.ArrDelay))
    # return
    XY
end

generate_xy (generic function with 1 method)

In [6]:
@time xy = generate_xy(yrtable)

  2.518327 seconds (19.38 M allocations: 1.961 GiB, 17.96% gc time)


6375689×26 Array{Float64,2}:
 1.0   837.0   -4.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  0.0   -3.0
 1.0   837.0   -1.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    2.0
 1.0   837.0   29.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    6.0
 1.0  1835.0   -2.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    7.0
 1.0  1835.0   18.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  -34.0
 1.0  1835.0   -4.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  0.0  -10.0
 1.0  1835.0   -4.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    4.0
 1.0  1835.0    0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   -2.0
 1.0  1835.0   -4.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  -23.0
 1.0   413.0    3.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   -1.0
 1.0   413.0   -4.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  0.0   19.0
 1.0   413.0   -3.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   16.0
 1.0   413.0    0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   1